# Classificação dados infomoney-data-clean

Nesse notebook vamos pegar os dados do arquivo 'infomoney-data-clean.csv' obtidos por meio de web scraping e classificaremos eles por meio do modelo disponível em https://huggingface.co/lucas-leme/FinBERT-PT-BR.

Para isso, começaremos instalando as dependências que precisaremos:

In [1]:
!pip install transformers 
!pip install sentencepiece 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 KB 66.9 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


Depois, precisamos importar os pacotes que utilizaremos.

In [2]:
import requests
import pandas as pd
import math as m
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-21 17:46:32.411618: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 17:46:32.575149: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-21 17:46:32.575180: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-21 

Após isso, precisamos ler os dados do nosso arquivo 'infomoney-data-clean.csv'. Para isso executamos o comando:

In [3]:
df = pd.read_csv('/work/DadosWebScrapping/infomoney-data-clean.csv')
df

,Unnamed: 0,text,title,subtitle,tags,date,author
0,0,Jader Filho afirmou que a ideia é subsidiar at...,"Minha Casa, Minha Vida: governo estuda aumenta...",Ministro das Cidades diz que intenção é chegar...,"['Economia', 'Minhas Finanças', 'Política', 'F...",2023-04-11 11:58:00,['Lucas Sampaio']
1,1,"Cabe ressaltar que, na noite da última segunda...",Taesa (TAEE11): o que a notícia sobre uma poss...,Análise da Genial não vê possível operação com...,"['Mercados', 'Ações', 'Energia Elétrica', 'Fol...",2023-04-11 15:35:00,['Felipe Moreira']
2,2,"“Eu não sou contra, pelo contrário, eu sempre ...",Prefeito de SP defende retirada do ISS da refo...,Ricardo Nunes (MDB) disse que não é contra a r...,"['Política', 'Reforma Tributária', 'Ricardo Nu...",2023-04-11 12:51:00,['Lucas Sampaio']
3,3,O Barômetro Econômico Global Coincidente encol...,"Barômetros globais caem em abril, puxados por ...","Para FGV/Ibre, resultados de abril refletem o ...","['Economia', 'Mercados', 'Barômetro', 'FGV/Ibre']",2023-04-11 13:05:00,['Estadão Conteúdo']
4,4,A diminuição da inadimplência foi impulsionada...,Inadimplência na capital paulista recua para 2...,Queda foi impulsionada pelos lares com renda i...,"['Minhas Finanças', 'Inadimplência', 'São Paulo']",2023-04-11 15:49:00,['Estadão Conteúdo']
...,...,...,...,...,...,...,...
11131,11131,"Itaúsa (ITSA4), por sua vez, aprovou aumento d...","CSN Mineração (CMIN3) anuncia R$ 2,4 bi em div...",Confira os destaques do noticiário corporativo...,"['Mercados', 'Negócios', 'Ambipar', 'Bradesco'...",2022-11-08 08:22:00,['Felipe Moreira']
11132,11132,As ações do banco alemão caíram pelo terceiro ...,"Ações do Deutsche Bank fecham em queda de 8,5%...",As ações do banco alemão caíram pelo terceiro ...,"['Economia', 'Mercados', 'Ações', 'Banco', 'Cr...",2023-03-24 15:28:00,['Equipe InfoMoney']
11133,11133,“O arcabouço fiscal vem ao encontro desse noss...,Tebet: arcabouço fiscal vai estabilizar dívida...,Ministra do Planejamento diz que texto é simpl...,"['Economia', 'Política', 'Ajuste Fiscal', 'Arc...",2023-03-27 16:08:00,['Marcos Mortari']
11134,11134,"De forma geral, o cenário econômico acaba tend...",Como as criptomoedas devem reagir às eleições ...,Os republicanos estão cada vez mais confiantes...,"['Mercados', 'Criptos', 'Eleições americanas',...",2022-11-08 09:31:00,['CoinDesk']


Para a classificação dos dados utilizaremos somente os títulos e subtítulos. Assim para selecionar somente a coluna dos títulos, executamos o comando:

In [4]:
titles = df.loc[:, df.columns == 'title']
titles

,title
0,"Minha Casa, Minha Vida: governo estuda aumenta..."
1,Taesa (TAEE11): o que a notícia sobre uma poss...
2,Prefeito de SP defende retirada do ISS da refo...
3,"Barômetros globais caem em abril, puxados por ..."
4,Inadimplência na capital paulista recua para 2...
...,...
11131,"CSN Mineração (CMIN3) anuncia R$ 2,4 bi em div..."
11132,"Ações do Deutsche Bank fecham em queda de 8,5%..."
11133,Tebet: arcabouço fiscal vai estabilizar dívida...
11134,Como as criptomoedas devem reagir às eleições ...


E para selecionar somente a coluna dos subtítulos, executamos o comando:

In [5]:
subtitles = df.loc[:, df.columns == 'subtitle']
subtitles

,subtitle
0,Ministro das Cidades diz que intenção é chegar...
1,Análise da Genial não vê possível operação com...
2,Ricardo Nunes (MDB) disse que não é contra a r...
3,"Para FGV/Ibre, resultados de abril refletem o ..."
4,Queda foi impulsionada pelos lares com renda i...
...,...
11131,Confira os destaques do noticiário corporativo...
11132,As ações do banco alemão caíram pelo terceiro ...
11133,Ministra do Planejamento diz que texto é simpl...
11134,Os republicanos estão cada vez mais confiantes...


Depois disso, precisamos configurar nosso modelo que classificará os textos. Para isso definimos qual é o caminho até os arquivos de nosso modelo:

In [6]:
path = '/work/ModeloHug'

Após isso, passamos o path para as classes AutoModelForSequenceClassification e AutoTokenizer para carregar o modelo pré-treinado e seu tokenizer associado. O tokenizer é o responsável pelas entradas do modelo. 

Além disso,  criamos um pipeline passando seu tipo ("sentiment-analysis") o modelo e o tokenizer. Um pipeline é um objeto que abstrai boa parte dos códigos complexos da biblioteca e facilita a utilização do modelo.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(path)

model = AutoModelForSequenceClassification.from_pretrained(path)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Para utilizar o modelo criado, basta digitar:

In [8]:
classifier("Frase que deseja analisar") 

[{'label': 'NEUTRAL', 'score': 0.6552619934082031}]

Exemplo real de uma notícia do mercado financeiro:

In [9]:
print(classifier("Ibovespa atinge recorde histórico e anima investidores"))

[{'label': 'POSITIVE', 'score': 0.7519307136535645}]


Como podemos ver, o resultado mais provável é de que a notícia é positiva. Agora vamos fazer isso para todos os títulos e subtítulos de notícias do nosso banco de dados. 

Começaremos pelos títulos. Para fazer isso criamos uma lista (results) para guardar os resultados do modelo. Depois criamos um laço que percorrerá todos os títulos, fará a classificação dele e adicionará o resultado dessa classificação na lista results. 

In [10]:
# Obs: A execução dessa célula demorou 1h e 20 minutos aproximadamente
resultsTitles = []
for x in range(11136):
    # print(x)
    if type(titles['title'][x]) is str:
        result = (classifier(titles['title'][x]))[0]['label']
        resultsTitles.append(result)
    else:
        resultsTitles.append('NaN')

Fazemos a mesma coisa com os subtítulos:

In [11]:
resultsSubTitles = []
for x in range(11136):
    # print(x)
    if type(subtitles['subtitle'][x]) is str:
        result = (classifier(subtitles['subtitle'][x]))[0]['label']
        resultsSubTitles.append(result)
    else:
        resultsSubTitles.append('NaN')

Após isso, adicionamos mais duas colunas ao nosso banco de dados inicial com os resultados obtidos.

In [15]:
df['classTitles'] = resultsTitles
df['classSubTitles'] = resultsSubTitles
df

,Unnamed: 0,text,title,subtitle,tags,date,author,classTitles,classSubTitles
0,0,Jader Filho afirmou que a ideia é subsidiar at...,"Minha Casa, Minha Vida: governo estuda aumenta...",Ministro das Cidades diz que intenção é chegar...,"['Economia', 'Minhas Finanças', 'Política', 'F...",2023-04-11 11:58:00,['Lucas Sampaio'],NEGATIVE,NEUTRAL
1,1,"Cabe ressaltar que, na noite da última segunda...",Taesa (TAEE11): o que a notícia sobre uma poss...,Análise da Genial não vê possível operação com...,"['Mercados', 'Ações', 'Energia Elétrica', 'Fol...",2023-04-11 15:35:00,['Felipe Moreira'],NEUTRAL,NEUTRAL
2,2,"“Eu não sou contra, pelo contrário, eu sempre ...",Prefeito de SP defende retirada do ISS da refo...,Ricardo Nunes (MDB) disse que não é contra a r...,"['Política', 'Reforma Tributária', 'Ricardo Nu...",2023-04-11 12:51:00,['Lucas Sampaio'],NEGATIVE,NEGATIVE
3,3,O Barômetro Econômico Global Coincidente encol...,"Barômetros globais caem em abril, puxados por ...","Para FGV/Ibre, resultados de abril refletem o ...","['Economia', 'Mercados', 'Barômetro', 'FGV/Ibre']",2023-04-11 13:05:00,['Estadão Conteúdo'],NEGATIVE,NEGATIVE
4,4,A diminuição da inadimplência foi impulsionada...,Inadimplência na capital paulista recua para 2...,Queda foi impulsionada pelos lares com renda i...,"['Minhas Finanças', 'Inadimplência', 'São Paulo']",2023-04-11 15:49:00,['Estadão Conteúdo'],NEGATIVE,NEGATIVE
...,...,...,...,...,...,...,...,...,...
11131,11131,"Itaúsa (ITSA4), por sua vez, aprovou aumento d...","CSN Mineração (CMIN3) anuncia R$ 2,4 bi em div...",Confira os destaques do noticiário corporativo...,"['Mercados', 'Negócios', 'Ambipar', 'Bradesco'...",2022-11-08 08:22:00,['Felipe Moreira'],NEUTRAL,NEUTRAL
11132,11132,As ações do banco alemão caíram pelo terceiro ...,"Ações do Deutsche Bank fecham em queda de 8,5%...",As ações do banco alemão caíram pelo terceiro ...,"['Economia', 'Mercados', 'Ações', 'Banco', 'Cr...",2023-03-24 15:28:00,['Equipe InfoMoney'],NEGATIVE,NEGATIVE
11133,11133,“O arcabouço fiscal vem ao encontro desse noss...,Tebet: arcabouço fiscal vai estabilizar dívida...,Ministra do Planejamento diz que texto é simpl...,"['Economia', 'Política', 'Ajuste Fiscal', 'Arc...",2023-03-27 16:08:00,['Marcos Mortari'],NEUTRAL,NEUTRAL
11134,11134,"De forma geral, o cenário econômico acaba tend...",Como as criptomoedas devem reagir às eleições ...,Os republicanos estão cada vez mais confiantes...,"['Mercados', 'Criptos', 'Eleições americanas',...",2022-11-08 09:31:00,['CoinDesk'],NEUTRAL,NEGATIVE


Para obter maior confiabilidade nos nossos dados, faremos uma comparação da classificação obtida pelo título e subtítulo de uma mesma notícia.

Para fazer isso, começaremos a selecionar as duas colunas referentes as classificações.

In [16]:
classification = df.loc[:,['classTitles', 'classSubTitles']]
classification

,classTitles,classSubTitles
0,NEGATIVE,NEUTRAL
1,NEUTRAL,NEUTRAL
2,NEGATIVE,NEGATIVE
3,NEGATIVE,NEGATIVE
4,NEGATIVE,NEGATIVE
...,...,...
11131,NEUTRAL,NEUTRAL
11132,NEGATIVE,NEGATIVE
11133,NEUTRAL,NEUTRAL
11134,NEUTRAL,NEGATIVE


Depois criamos um laço que passa por todas as 'linhas' da nossa tabela anterior, compara as duas classificações e guarda essa informação numa lista (resultsMatch). Além disso ela guarda a quantidade de cada resultado (de Miss, Match, NaN).

In [19]:
qtdNaN = 0
qtdMatch = 0
qtdMiss = 0
resultsMatch = []
for x in range(11136):
    if((type(classification['classTitles'][x]) is str) and (type(classification['classSubTitles'][x]) is str)):
        if(classification['classTitles'][x] == "NaN" or classification['classSubTitles'][x] == "NaN"):
            qtdNaN += 1
            resultsMatch.append("NaN")
        elif(classification['classTitles'][x] == classification['classSubTitles'][x]):
            qtdMatch += 1
            resultsMatch.append("Match")
        else:
            qtdMiss += 1
            resultsMatch.append("Miss")
    else:
        qtdNaN += 1
        resultsMatch.append("NaN")
total = qtdMatch + qtdMiss + qtdNaN
tabela = [["","Match", "Miss", "NaN", "Total"], ["Quantidade", qtdMatch , qtdMiss, qtdNaN, total], ["Proporção", "%.2f%c" % ((qtdMatch/total * 100), "%"), "%.2f%c" % ((qtdMiss/total * 100), "%"), "%.2f%c" % ((qtdNaN/total * 100), "%"), "%.0f%c" % ((total/total * 100), "%")]]
matchBanco = pd.DataFrame(tabela)
matchBanco

,0,1,2,3,4
0,,Match,Miss,NaN,Total
1,Quantidade,6760,4376,0,11136
2,Proporção,60.70%,39.30%,0.00%,100%


Depois disso, precisamos guardar as informações de match em nosso banco. Para fazer isso adicionamos mais uma coluna em nossa tabela

In [20]:
df['Match'] = resultsMatch
df

,Unnamed: 0,text,title,subtitle,tags,date,author,classTitles,classSubTitles,Match
0,0,Jader Filho afirmou que a ideia é subsidiar at...,"Minha Casa, Minha Vida: governo estuda aumenta...",Ministro das Cidades diz que intenção é chegar...,"['Economia', 'Minhas Finanças', 'Política', 'F...",2023-04-11 11:58:00,['Lucas Sampaio'],NEGATIVE,NEUTRAL,Miss
1,1,"Cabe ressaltar que, na noite da última segunda...",Taesa (TAEE11): o que a notícia sobre uma poss...,Análise da Genial não vê possível operação com...,"['Mercados', 'Ações', 'Energia Elétrica', 'Fol...",2023-04-11 15:35:00,['Felipe Moreira'],NEUTRAL,NEUTRAL,Match
2,2,"“Eu não sou contra, pelo contrário, eu sempre ...",Prefeito de SP defende retirada do ISS da refo...,Ricardo Nunes (MDB) disse que não é contra a r...,"['Política', 'Reforma Tributária', 'Ricardo Nu...",2023-04-11 12:51:00,['Lucas Sampaio'],NEGATIVE,NEGATIVE,Match
3,3,O Barômetro Econômico Global Coincidente encol...,"Barômetros globais caem em abril, puxados por ...","Para FGV/Ibre, resultados de abril refletem o ...","['Economia', 'Mercados', 'Barômetro', 'FGV/Ibre']",2023-04-11 13:05:00,['Estadão Conteúdo'],NEGATIVE,NEGATIVE,Match
4,4,A diminuição da inadimplência foi impulsionada...,Inadimplência na capital paulista recua para 2...,Queda foi impulsionada pelos lares com renda i...,"['Minhas Finanças', 'Inadimplência', 'São Paulo']",2023-04-11 15:49:00,['Estadão Conteúdo'],NEGATIVE,NEGATIVE,Match
...,...,...,...,...,...,...,...,...,...,...
11131,11131,"Itaúsa (ITSA4), por sua vez, aprovou aumento d...","CSN Mineração (CMIN3) anuncia R$ 2,4 bi em div...",Confira os destaques do noticiário corporativo...,"['Mercados', 'Negócios', 'Ambipar', 'Bradesco'...",2022-11-08 08:22:00,['Felipe Moreira'],NEUTRAL,NEUTRAL,Match
11132,11132,As ações do banco alemão caíram pelo terceiro ...,"Ações do Deutsche Bank fecham em queda de 8,5%...",As ações do banco alemão caíram pelo terceiro ...,"['Economia', 'Mercados', 'Ações', 'Banco', 'Cr...",2023-03-24 15:28:00,['Equipe InfoMoney'],NEGATIVE,NEGATIVE,Match
11133,11133,“O arcabouço fiscal vem ao encontro desse noss...,Tebet: arcabouço fiscal vai estabilizar dívida...,Ministra do Planejamento diz que texto é simpl...,"['Economia', 'Política', 'Ajuste Fiscal', 'Arc...",2023-03-27 16:08:00,['Marcos Mortari'],NEUTRAL,NEUTRAL,Match
11134,11134,"De forma geral, o cenário econômico acaba tend...",Como as criptomoedas devem reagir às eleições ...,Os republicanos estão cada vez mais confiantes...,"['Mercados', 'Criptos', 'Eleições americanas',...",2022-11-08 09:31:00,['CoinDesk'],NEUTRAL,NEGATIVE,Miss


Por fim, criamos um novo arquivo para salvar o novo banco.

In [21]:
df.to_csv('/work/DadosClassificados/infomoney-data-cleanCLASSIFIED.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4246ec96-5bda-435e-947c-91f08e9f7576' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>